# Chemical kinetics: the basics

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Nov 16th, 2022</p>

In this first tutorial about chemical kinetics modeling using Reaktoro, we simulate the dissolution of a 1 cm³ cube of halite crystal, NaCl(s), in pure water at 25 °C and 1 bar. We make the following assumptions:

1. The crystal remains a cube as it dissolves and its surface area over time can be computed as *current crystal volume* × 6 cm²/cm³.
2. The reactions among aqueous species are much faster than that of the crystal dissolution. As a result, all aqueous species are assumed in equilibrium at all times.

```{note}
There is no need to define the aqueous reactions because Reaktoro uses a Gibbs energy minimization algorithm to compute the equilibrium state of all species in the system that are not restricted by chemical kinetics constraints.
```


Before anything else, let's import the `reaktoro` Python package:

In [25]:
from reaktoro import *

For this simulation, we need a *reaction rate model* for the reaction NaCl(s) = Na<sup>+</sup> + Cl<sup>-</sup>. Before we define it next, note that NaCl(s) will be referred as `Halite`, which is the name of this mineral in the PHREEQC database `phreeqc.dat` used in this tutorial. 

In [26]:
# The surface area of a cube per volume (in m2/m3)
Abar = 6.0

def ratefn(props: ChemicalProps):
    aprops = AqueousProps(props)  # we need an AqueousProps object to compute the saturation ratio of the mineral
    k0 = pow(10.0, -0.21)  # the reaction rate constant at 25 °C from Palandri and Kharaka (2004)
    q = props.phaseProps("Halite").volume()  # the current volume of the mineral (in m3)
    Omega = aprops.saturationRatio("Halite")  # the current saturation ratio of the mineral Ω = IAP/K
    return q * Abar * k0 * (1 - Omega)  # (1 - Ω) is the current measure of disequilibrium! Ω < 1 implies mineral is undersaturated

```{note}
A **reaction rate model** in Reaktoro is just a function that *receives* an object of class {{ChemicalProps}} (i.e., `props`) containing the current thermodynamic and chemical properties of the system and *returns* the computed reaction rate.
```

We'll now define our chemical system for this simulation, which contains an aqueous solution, a mineral phase, and a reaction:

In [27]:
db = PhreeqcDatabase("phreeqc.dat")

system = ChemicalSystem(db,
    AqueousPhase("H2O H+ OH- Na+ Cl- NaOH").set(ActivityModelHKF()),
    MineralPhase("Halite"),
    GeneralReaction("Halite = Na+ + Cl-").setRateModel(ratefn)  # define the reaction and set its reaction rate model!
)

```{tip}
Given that all aqueous species are assumed to react much faster than the mineral, we could also have defined the above reaction using just the mineral name `GeneralReaction("Halite")`. Try this and check the results, which should be very similar to the ones we get here.

With this alternative reaction definition, Reaktoro will use the assigned reaction rate model to determine how much the mineral dissolves over time and will use chemical equilibrium and mass conservation equations (because chemical elements and electrical charge are preserved in this problem!) to determine how the amounts of aqueous species change as the mineral reacts.
```

The next step is to define the initial condition of the system: 1 kg of pure water and 1 cm³ of halite mineral at 25 °C and 1 bar:

In [28]:
state = ChemicalState(system)
state.temperature(25.0, "C")
state.pressure(1.0, "bar")
state.set("H2O", 1.0, "kg")
state.scalePhaseVolume("Halite", 1.0, "cm3")  # start with 1 cm3 cube of halite crystal

Next, we'll define the chemical kinetics solver and a few other things needed for the simulation (detailed in the comments), and then we'll start the time stepping procedure to compute a sequence of chemical states, each obtained by letting the system react for a specified length of time:

In [29]:
solver = KineticsSolver(system)  # the chemical kinetics solver

table = Table()  # used to create table of data for later output and plotting

dt = 60.0  # time step (in seconds)

# Initiate the time stepping for the kinetics modeling
for i in range(501):
    result = solver.solve(state, dt)  # compute the chemical state of the system after it reacted for given time length

    assert result.succeeded(), f"Calculation did not succeed at time step #{i}."

    props = state.props()  # get the current thermodynamic and chemical properties of the system

    table.column("Time")   << i*dt / 60  # from seconds to minutes
    table.column("Halite") << props.phaseProps("Halite").volume() * 1e+6  # from m3 to cm3
    table.column("Na+")    << props.speciesAmount("Na+")  # in mol
    table.column("Cl-")    << props.speciesAmount("Cl-")  # in mol

The collected data above can be saved to a file:

In [30]:
table.save("data.txt")

And it looks like this:

In [31]:
print(table)

   Time |    Halite |         Na+ |         Cl-
0.00000 |  0.994020 | 0.000220647 | 0.000220647
1.00000 |  0.988077 | 0.000439974 | 0.000439974
2.00000 |  0.982168 | 0.000657991 | 0.000657991
3.00000 |  0.976296 | 0.000874703 | 0.000874703
4.00000 |  0.970458 |  0.00109012 |  0.00109012
5.00000 |  0.964655 |  0.00130425 |  0.00130425
6.00000 |  0.958887 |  0.00151710 |  0.00151710
7.00000 |  0.953153 |  0.00172867 |  0.00172867
8.00000 |  0.947454 |  0.00193898 |  0.00193898
9.00000 |  0.941788 |  0.00214803 |  0.00214803
10.0000 |  0.936157 |  0.00235584 |  0.00235584
11.0000 |  0.930559 |  0.00256240 |  0.00256240
12.0000 |  0.924995 |  0.00276772 |  0.00276772
13.0000 |  0.919464 |  0.00297182 |  0.00297182
14.0000 |  0.913966 |  0.00317470 |  0.00317470
15.0000 |  0.908501 |  0.00337636 |  0.00337636
16.0000 |  0.903068 |  0.00357682 |  0.00357682
17.0000 |  0.897668 |  0.00377608 |  0.00377608
18.0000 |  0.892301 |  0.00397414 |  0.00397414
19.0000 |  0.886965 |  0.00417103 |  0.0

We can also make plots, using the plotting library `reaktplot` that was developed for use with Reaktoro:

In [32]:
from reaktplot import *

fig1 = Figure()
fig1.title("AQUEOUS SPECIES AMOUNTS OVER TIME")
fig1.xaxisTitle("Time [minute]")
fig1.yaxisTitle("Amount [mol]")
fig1.drawLine(table["Time"], table["Na+"], "Na<sup>+</sup>")
fig1.drawLine(table["Time"], table["Cl-"], "Cl<sup>-</sup>")
fig1.show()

In [33]:
fig2 = Figure()
fig2.title("CALCITE VOLUME OVER TIME")
fig2.xaxisTitle("Time [minute]")
fig2.yaxisTitle("Volume [m<sup>3</sup>]")
fig2.drawLine(table["Time"], table["Halite"], "Halite")
fig2.show()

You can also save these figures to one of several file formats (PNG, JPEG, WEBP, SVG, PDF, EPS, or HTML) and add them to your publication:

In [34]:
fig1.save("fig1.pdf")
fig2.save("fig2.pdf")

That's it. This tutorial covered the basics of chemical kinetics modeling in Reaktoro. Continue reading to learn more!